In [4]:
from dotenv import load_dotenv
from smolagents import LiteLLMModel
import os
from huggingface_hub import InferenceClient

load_dotenv()
HF_Token = os.getenv("HF_TOKEN")



model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5:latest",
    api_base="http://localhost:11434",
    num_ctx=8192,
)

In [5]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, InferenceClientModel

class PartyPlanningRetrieverTool(Tool):
    name = "party_planning_retriever"
    description = "Uses semantic search to retrieve relevant party planning ideas for Alfred’s superhero-themed party at Wayne Manor."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a query related to party planning or superhero themes.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5  # Retrieve the top 5 documents
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

# Initialize the agent
agent = CodeAgent(tools=[party_planning_retriever], model=model)

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options.      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5:latest ─────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  party_ideas = party_planning_retriever(query="luxury superhero-themed party")                                    
  print(party_ideas)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

Out: None

[Step 1: Duration 92.03 seconds| Input tokens: 2,065 | Output tokens: 59]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer_variable = (                                                                                        
      "For decoration, consider a masquerade ball with gold accents and velvet curtains.\n"                        
      "\n"                                                                                                         
      "Interactive experiences could include VR simulations or themed games for guests to engage in.\n"            
      "\n"                                                                                                         
      "Decorate the venue with iconic superhero logos and projections of Gotham and other superhero cities.\n"     
      "\n"                                                                                                         
      "Catering options should include dishes named after superheroes, such as 'The Hulk's Green Smoothie' and     
  'Iron Man's Power Steak'.\n"                                                                                     
      "\n"                                                                                                         
      "A professional DJ can play themed music for superheroes like Batman and Wonder Woman."                      
  )                                                                                                                
  final_answer(final_answer_variable)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: For decoration, consider a masquerade ball with gold accents and velvet curtains.

Interactive experiences could include VR simulations or themed games for guests to engage in.

Decorate the venue with iconic superhero logos and projections of Gotham and other superhero cities.

Catering options should include dishes named after superheroes, such as 'The Hulk's Green Smoothie' and 'Iron Man's
Power Steak'.

A professional DJ can play themed music for superheroes like Batman and Wonder Woman.

[Step 2: Duration 18.02 seconds| Input tokens: 4,404 | Output tokens: 241]

For decoration, consider a masquerade ball with gold accents and velvet curtains.

Interactive experiences could include VR simulations or themed games for guests to engage in.

Decorate the venue with iconic superhero logos and projections of Gotham and other superhero cities.

Catering options should include dishes named after superheroes, such as 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak'.

A professional DJ can play themed music for superheroes like Batman and Wonder Woman.
